### __Paquetes__

In [33]:
import Tools as t
import pandas as pd
import numpy as np
from notion_client import Client
from datetime import datetime
import re
import emoji
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog

# Formularios.
from Formulario import Explorer_Input_Title, Filetypes_Tuples_Input

# P_Start.
import P_Start

# Módulos personalizados.
import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Universidad/Patricionog/Datos/Paquetes/')
import Foragio as fg
import Excelio as ex
import Framio as fr
import Stringio as st
import Listio as ls
import Graphio as gr
import Notio as nt
import Highlightio as hg

### __Variables globales__

In [34]:
# Páginas.

Help_Valid_Txt = 'www.exhurbia.com/valid-txt'

In [35]:
# Notion.

Token = 'secret_NYOvwjAc63k8bhNv0zpfaeUX1S7dk5ilgek1hoWXKkR'
Client = Client(auth=Token)

### __Etapa 1. Pantalla de inicio (P_Start)__

In [36]:
File_Path = P_Start.P_Start()

NameError: name 'Explorer_Input_Title' is not defined

### __Etapa 2. Pantalla de bloques (P_Display)__

##### 1. Dataframe de notas 

In [40]:
#if Default_Match == False:

# df total del recorte.
df = hg.Build_Df_Of_Highlights(File_Path)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

df

ValueError: time data '2022.0-02-14.0, 00:00:00' does not match format '%Y-%m-%d %H:%M:%S'

##### 2. Ordenamiento de los bloques por el usuario

In [41]:
# Estas dos opciones debería elegirlas el usuario.

# Opciones: ['Author', 'Book', 'Date']
Order_By = 'Author'
Ascending = False

df = fr.Sort_Columns(df, Order_By)

##### 3. Agrupamiento por condición que ponga el usuario 

In [42]:
# Opciones: [None, 'Author', 'Book', 'Day_Of_Week', 'Date', 'Month', 'Year']
Group_By = 'Book'

if Group_By:
    Clippings = fr.Divide_DataFrames_By_Column(df, Group_By)

##### 4. Combinación de Blocks

##### 5. Eliminación de Blocks

##### 6. Dividir Blocks

##### 7. Cambio de nombre de Clippings

##### 8. Eliminar Clippings

### __3. Procesamiento del texto (H_Proccesing)__

In [43]:
# Dejar pipí cucú los highlights.

df.loc[:, 'Highlight'] = df['Highlight'].apply(t.Procesar_Texto)

### __Etapa 4. Integración con Notion (P_Notion)__

In [44]:
# Elecciones del usuario.

Database_ID = '129a53d444454d63820814429c4fb691'

In [45]:
# Base de datos con autores e ID de las páginas en Notion dónde se pegan sus notas.
Base_Matcheos = []

In [46]:
# Extraer autores del recorte.
Autores = t.Extraer_Datos_Nota(File_Path, 'Ambos')

In [47]:
Autores = t.Extraer_Datos_Nota(File_Path, 'Autores')

In [48]:
# Matchear autores nuevos.
Base_Matcheos = t.Matcheo_Notas_Con_Base_Notion(Token, Autores, Database_ID, Base_Matcheos)

# Esta función vincula cada página de la base de datos con un elemento de la nota (libro o autor).
# Hay que generalizarla, porque solo funciona con autores. Tiene que funcionar con libros. 
# Y ver si hay alguna forma más.

Error al intentar acceder a la base de datos: 'str' object has no attribute 'databases'


TypeError: object of type 'NoneType' has no len()

In [9]:
# Lista de diccionarios con el autor y sus frases.

Lista_Diccionarios_Autores = t.Lista_De_Diccionarios_Por_Elemento_Padre(df, Autores, 'Autor', ['Libro', 'Highlight'])

In [10]:
# Columnas nuevas en el df.

df['ID_Autor'] = ''
df['ID_Libro'] = ''
df['Emoji'] = ''

In [11]:
# Armamos un df para los libros con sus respectivos ID.

df_Libros = pd.DataFrame()

for Autor in Autores:
    # Extraemos los ID de cada autor.
    for Match in Base_Matcheos:
        if Match['Autor'] == Autor:
            ID = Match['ID']

    # Formamos un df con las frases de cada autor.
    df_Frases_Autor = df[df['Autor'] == Autor]  

    # Formamos un df con los libros del autor.
    df_Libros_Autor = t.Obtener_IDs_y_Contenido_Bloques(Token, ID)

    # Eliminamos el número y los paréntesis.
    df_Libros_Autor['Contenido'] = df_Libros_Autor['Contenido'].apply(lambda x: t.Eliminar_Texto_Entre_Caracteres(x, '(', ')').strip())

    # Concatenar df1 y df2 verticalmente
    df_Libros = pd.concat([df_Libros, df_Libros_Autor], ignore_index=True)

    # Reordenar columnas.
    df_Libros = df_Libros[['Contenido', 'ID']]

In [12]:
# Pegar ID_Libro e ID_Autor en df.

# Creamos este df para poder transferir datos al df.
df_Matcheos = pd.DataFrame(Base_Matcheos)

df = t.Comparar_Df_y_Copiar(df, df_Libros, 'Libro', 'ID_Libro', 'Contenido', 'ID')
df = t.Comparar_Df_y_Copiar(df, df_Matcheos, 'Autor', 'ID_Autor', 'Autor', 'ID')

df[0:5]

,Autor,Libro,Pagina,Dia de la semana,Dia,Mes,Año,Hora,Highlight,ID_Autor,ID_Libro,Emoji
0,Lucio Anneo Séneca,Obras completas,108,Martes,18,Junio,2024,3:24,sobre los relojes públicos solía grabarse la i...,3b753b78-187f-414f-a78c-a9d106cca7fb,d96488ee-b603-42cb-97ed-8a3d8014f3d6,
1,Lucio Anneo Séneca,Obras completas,49,Martes,18,Junio,2024,3:25,«Es ridículo que te apresures a la muerte por ...,3b753b78-187f-414f-a78c-a9d106cca7fb,d96488ee-b603-42cb-97ed-8a3d8014f3d6,
2,Lucio Anneo Séneca,Séneca,49,Martes,18,Junio,2024,3:27,«¿Qué hay tan ridículo como suspirar por la mu...,3b753b78-187f-414f-a78c-a9d106cca7fb,,
3,Lucio Anneo Séneca,Séneca,49,Martes,18,Junio,2024,3:27,se nos ha de exhortar y fortalecer para este d...,3b753b78-187f-414f-a78c-a9d106cca7fb,,
4,Lucio Anneo Séneca,Séneca,50,Martes,18,Junio,2024,3:29,"»Es decir: me despertaré, dormiré; comeré, ten...",3b753b78-187f-414f-a78c-a9d106cca7fb,,


### __5. Generación del Output (P_End)__

In [15]:
# Generar pdf.
from fpdf import FPDF

In [ ]:
# Nombre del PDF final.
PDF_Name = 'PDF'

In [16]:
# Variables.

New_Font = 'ArialUnicode'
Pathway_New_Font = 'Fonts/arial-unicode-ms.ttf'

In [17]:
# Definir fuente del Highlight.

Font = New_Font
Fontsize = 12
Style = ''

In [18]:
# Propiedades de la celda.

Cell_Weight = 190
Cell_Height = 5
Cell_Align = 'L'

# Salto de línea después de la celda.
Cell_Line_Break = True

In [19]:
# Definir orden del highlight.

# Pueden elegir entre 6 órdenes.

# 1) 
# Highlight.
# Libro (Autor).

# 2) 
# Highlight.
# Autor (Libro).

# 3) 
# Highlight.
# Autor.

# 4) 
# Highlight.
# Libro.

# 5) 
# Highlight.
# Autor.
# Libro.

# 5) 
# Highlight.
# Libro.
# Autor.

Highlight_Style = 1

In [20]:
# Definir fuente de autor y libro.

# Autor.
Author_Font = Font
Author_Fontsize = 10
Autor_Align = 'L'
Author_Style = 'I'

# Libro.
Book_Font = Font
Book_Fontsize = 10
Book_Align = 'L'
Book_Style = 'I'

In [21]:
# Definir espacio.
Space_Between_Highlights = 8
Space_Between_Highlight_And_Author_Or_Book = 1.5

In [22]:
# Crea un objeto llamado 'PDF', instancia de la clase FPDF.
PDF = FPDF() 

In [23]:
# Añadir fuente al pdf para que pueda leer caracteres especiales.
PDF.add_font(New_Font, '', Pathway_New_Font, uni=True) # Normal
PDF.add_font(New_Font, 'I', Pathway_New_Font, uni=True) # Italic
PDF.add_font(New_Font, 'B', Pathway_New_Font, uni=True) # Bold
PDF.add_font(New_Font, 'BI', Pathway_New_Font, uni=True) # Bold Italic

C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\1237765908.py:2: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  PDF.add_font(New_Font, '', Pathway_New_Font, uni=True) # Normal
C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\1237765908.py:3: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  PDF.add_font(New_Font, 'I', Pathway_New_Font, uni=True) # Italic
C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\1237765908.py:4: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  PDF.add_font(New_Font, 'B', Pathway_New_Font, uni=True) # Bold
C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\1237765908.py:5: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  PDF.add_font(New_Font, 'BI', Pathway_New_Font, uni=True) # Bold Italic


In [24]:
# Añadir una página al documento.
PDF.add_page()  

In [25]:
# Establecer la fuente.
PDF.set_font(Font, size=Fontsize, style=Style)  

In [26]:
for Index, Row in df.iterrows():
    # Agregar celda con la nota.
    PDF.set_font(Font, Style, Fontsize)
    PDF.multi_cell(Cell_Weight, Cell_Height, text=Row['Highlight'], ln=Cell_Line_Break, align=Cell_Align)

    # Espacio con la línea del autor o libro.
    PDF.ln(Space_Between_Highlight_And_Author_Or_Book)

    # AGREGAR LOS OTROS ESTILOS DESPUÉS.
    if Highlight_Style == 1:

        # Cambiar fuente, tamaño y estilo solo para esta celda
        PDF.set_font(Font, Author_Style, Author_Fontsize)
        PDF.cell(Cell_Weight, Cell_Height, text=f'{Row["Libro"]} ({Row["Autor"]})', ln=Cell_Line_Break, align=Autor_Align)
        
    # Espacio con el otro highlight.
    PDF.ln(Space_Between_Highlights)


C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\2176654088.py:4: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  PDF.multi_cell(Cell_Weight, Cell_Height, text=Row['Highlight'], ln=Cell_Line_Break, align=Cell_Align)
C:\Users\tomas\AppData\Local\Temp\ipykernel_17568\2176654088.py:14: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  PDF.cell(Cell_Weight, Cell_Height, text=f'{Row["Libro"]} ({Row["Autor"]})', ln=Cell_Line_Break, align=Autor_Align)


In [27]:
# Guarda el documento PDF en un archivo
PDF.output(f'C:/Users/tomas/Downloads/{PDF_Name}.pdf')  